In [31]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from tqdm import tqdm
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout,LSTM,Masking
from keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, Dropout, Dense,LSTM
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input , Conv2D , MaxPooling2D , Dropout , concatenate , UpSampling2D, MaxPooling3D, Conv3D, Reshape, Activation
from tensorflow.keras import models
from tensorflow.keras import losses
from keras.layers import Dense, Conv3D , MaxPool3D , Flatten , Dropout,LSTM,BatchNormalization
import tifffile

import os 
from skimage import io
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from skimage.color import gray2rgb
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

In [32]:
from Models import *
from SDMUtils import *

In [33]:
# Choose architecture 
architecture = 'ViViT'  # 'ViViT' or 'LSTM' 

# Choose number of timepoints, planes 
timespace = 1
plane=8

# Choose the model directory
dir = "Models Directory"

# DO NOT MODIFY FOR REPRODUCIBILITY
seed= 12

In [34]:
model_name = "ModelName.h5"


## Load data

In [20]:
# Current directory 
directory = os.getcwd()
print(directory)

# Load Events
folders = os.walk(directory+'/data/Events')
data = [x[0] for x in folders]
data = data[1:]

events=[]
for i in tqdm(range(len(data))):
    events+= [np.transpose(np.reshape([(
        io.imread(data[i]+'/'+f)-np.min(io.imread(data[i]+'/'+f)))/(np.max(io.imread(data[i]+'/'+f))-np.min(io.imread(data[i]+'/'+f))) # normalize pixels between 0 and 1
        for f in sorted(os.listdir(data[i]))[-8*timespace:]],  # take the 8 planes of the last t timepoints
        (-1,8,352,352)), # reshape to split planes and timepoints ; shape=(timepoints,planes,im_shape,im_shape)
        (0,2,3,1)) # reshape to have (timepoints,im_shape,im_shape,plane)
        ] 
    
# Load Non-Events
ne = os.walk(directory+'/data/Non-Events')
data = [x[0] for x in ne]
non_events=[]
data = data[1:]
for i in tqdm(range(len(data))):
    non_events+= [np.transpose(np.reshape([(
        io.imread(data[i]+'/'+f)-np.min(io.imread(data[i]+'/'+f)))/(np.max(io.imread(data[i]+'/'+f))-np.min(io.imread(data[i]+'/'+f))) # normalize pixels between 0 and 1
        for f in sorted(os.listdir(data[i]))[-8*timespace:]],  # take the 8 planes of the last t timepoints
        (-1,8,352,352)), # reshape to split planes and timepoints ; shape=(timepoints,planes,im_shape,im_shape)
        (0,2,3,1)) # reshape to have (timepoints,im_shape,im_shape,plane)
        ]     

C:\Users\kibrahim\Downloads\Code\Python\Deep Learning 2022\SMART


100%|██████████████████████████████████████████████████████████████████████████████████| 68/68 [00:01<00:00, 45.88it/s]


In [21]:
try : 
    x_train, y_train, x_test, y_test = prepare_sets(events,non_events,seed,timespace,plane)
except Error as e:
    print(e)

In [37]:
if architecture =='ViViT' : 
    model = load_vivit(dir +"/"+model_name,timespace,plane)
    all_layers=None
elif architecture == 'LSTM' : 
    model = load_lstm(timespace,plane,dir +"/"+model_name)
    all_layers = [layer.name for layer in reversed(model.layers) if len(layer.output_shape) ==5 and (layer.__class__.__name__ == 'TimeDistributed' and layer.layer.__class__.__name__ == 'Conv2D' )]
else :
    model = load_vgg(plane, dir +"/"+model_name)
    all_layers = [layer.name for layer in reversed(model.layers) if len(layer.output_shape) ==5 and (layer.__class__.__name__ == 'TimeDistributed' and layer.layer.__class__.__name__ == 'Conv2D' )]
    x_test = x_test.reshape([x_test.shape[0], 352, 352, plane])

## Analyse test set

In [29]:
accuracy=plot_1tstep(x_test,y_test,plane,architecture,model,dir,'test_set','image',all_layers,print_tif=False)